In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [8]:
%%capture
%run "7. Evaluación del modelo robusto.ipynb"

In [9]:
# Diccionarios para guardar datos.

# Variables significativas.
Variables_Que_Explican = {}

In [10]:
# Resultados de los modelos.
Resultados_Modelos = {}

In [11]:
# Evaluaciones de los modelos.
Evaluaciones_Modelos = {}

In [12]:
# Ejecutar el modelo robusto para todas las variables dependientes.
for i, Variable in enumerate(Variables_Dependientes['Ballotage']):
    
    print(f"\n🔄 Procesando variable {i+1}/{len(Variables_Dependientes['Ballotage'])}: {Variable}")
    
    try:

        # Limpiar y estructurar datos eliminando observaciones sin variable dependiente e imputar faltantes con mediana/moda.
        Variables_X_Prueba, Variable_Y_Prueba = Limpiar_Y_Estructurar_Datos_Para_Modelado(
                                                df_Ballotage, 
                                                Variable, 
                                                Variables_Independientes)
        
        # Transformar variables booleanas a representación entera binaria (0/1) para compatibilidad con statsmodels.
        Variables_X_Prueba = Codificar_Variables_Booleanas_A_Numericas(Variables_X_Prueba)
        
        # Codificar variables categóricas restantes a formato numérico usando Label Encoding como respaldo.
        Variables_X_Prueba = Codificar_Variables_Categoricas_A_Numericas(Variables_X_Prueba)
        
        # Ejecutar modelado robusto metodológicamente sólido con eliminación secuencial, detección VIF y outliers.
        Resultado_Robusto = Modelo_Lineal_Robusto(
            Variables_X=Variables_X_Prueba, 
            Variable_Y=Variable_Y_Prueba, 
            Nombre_Variable=Variable,
            Criterio_Eliminacion='aic',  # Usar AIC para selección de modelos
            Umbral_VIF=5.0,              # Umbral estándar para multicolinealidad
            Alpha_Significancia=0.05,    # Nivel de significancia tradicional
            Detectar_Outliers=True       # Activar detección de observaciones influyentes
        )
        
        # Almacenar resultados completos del modelo robusto con todas las métricas y diagnósticos.
        Resultados_Modelos[Variable] = Resultado_Robusto
        
        # Interpretar y evaluar modelo con análisis comprensible de validez y recomendaciones.
        Evaluacion = Evaluar_Modelo_Robusto(
            Resultados_Modelo=Resultado_Robusto, 
            Mostrar_Detalles_Tecnicos=False  # True para diagnósticos técnicos detallados
        )
        
        # Almacenar evaluación estructurada con puntuación de confianza y recomendaciones.
        Evaluaciones_Modelos[Variable] = Evaluacion
        
        # Extraer variables predictoras con efectos estadísticamente significativos del modelo final.
        if Resultado_Robusto is not None:
            Variables_Significativas = Resultado_Robusto['Variables_Significativas']
            
            if Variables_Significativas:
                Variables_Que_Explican[Variable] = Variables_Significativas
                print(f"   Variables significativas: {len(Variables_Significativas)}")
                print(f"   R² ajustado: {Resultado_Robusto['R_Cuadrado_Ajustado']:.3f}")
            else:
                Variables_Que_Explican[Variable] = ['-']
                print(f"   ⚠️ No se encontraron variables significativas")
        else:
            Variables_Que_Explican[Variable] = ['-']
            print(f"   ❌ Modelo no válido - no se pudo ajustar")
            
        print(f"✅ {Variable} procesado exitosamente")
        
    except Exception as e:
        print(f"❌ Error procesando {Variable}: {str(e)}")
        print(f"   Tipo de error: {type(e).__name__}")
        
        # Registrar falla completa con valores nulos para mantener consistencia entre diccionarios.
        Resultados_Modelos[Variable] = None
        Evaluaciones_Modelos[Variable] = None
        Variables_Que_Explican[Variable] = ['-']
        
        # Continuar procesamiento masivo sin interrumpir por errores individuales.
        continue

# Generar reporte comprehensivo del procesamiento masivo de variables dependientes.
print(f"\n📊 RESUMEN FINAL DEL PROCESAMIENTO MASIVO:")
print("="*60)

Total_Variables = len(Resultados_Modelos)
Modelos_Validos = sum(1 for r in Resultados_Modelos.values() if r is not None)
Evaluaciones_Completas = sum(1 for e in Evaluaciones_Modelos.values() if e is not None)

print(f"Variables dependientes procesadas: {Total_Variables}")
print(f"Modelos robustos válidos: {Modelos_Validos} ({Modelos_Validos/Total_Variables*100:.1f}%)")
print(f"Evaluaciones completadas: {Evaluaciones_Completas} ({Evaluaciones_Completas/Total_Variables*100:.1f}%)")

# Análisis de calidad de los modelos válidos.
if Modelos_Validos > 0:
    print(f"\n📈 ANÁLISIS DE CALIDAD DE MODELOS VÁLIDOS:")
    
    R2_Valores = [r['R_Cuadrado_Ajustado'] for r in Resultados_Modelos.values() if r is not None]
    Variables_Sig_Totales = [len(r['Variables_Significativas']) for r in Resultados_Modelos.values() if r is not None]
    
    print(f"R² ajustado promedio: {np.mean(R2_Valores):.3f}")
    print(f"R² ajustado máximo: {np.max(R2_Valores):.3f}")
    print(f"R² ajustado mínimo: {np.min(R2_Valores):.3f}")
    print(f"Variables significativas promedio: {np.mean(Variables_Sig_Totales):.1f}")
    
    # Identificar los mejores modelos.
    Mejores_Modelos = []
    for Variable, Resultado in Resultados_Modelos.items():
        if Resultado is not None and Resultado['R_Cuadrado_Ajustado'] >= 0.30:
            Mejores_Modelos.append((Variable, Resultado['R_Cuadrado_Ajustado']))
    
    if Mejores_Modelos:
        Mejores_Modelos.sort(key=lambda x: x[1], reverse=True)
        print(f"\n🏆 MEJORES MODELOS (R² ≥ 0.30):")
        for i, (Variable, R2) in enumerate(Mejores_Modelos[:5], 1):
            N_Vars = len(Resultados_Modelos[Variable]['Variables_Significativas'])
            print(f"  {i}. {Variable}: R² = {R2:.3f} ({N_Vars} variables significativas)")
    
    # Análisis de variables más influyentes.
    Contador_Variables = {}
    for Resultado in Resultados_Modelos.values():
        if Resultado is not None:
            for Var_Sig in Resultado['Variables_Significativas']:
                Contador_Variables[Var_Sig] = Contador_Variables.get(Var_Sig, 0) + 1
    
    if Contador_Variables:
        Variables_Frecuentes = sorted(Contador_Variables.items(), key=lambda x: x[1], reverse=True)
        print(f"\n🎯 VARIABLES PREDICTORAS MÁS FRECUENTES:")
        for i, (Variable, Frecuencia) in enumerate(Variables_Frecuentes[:10], 1):
            Porcentaje = (Frecuencia / Modelos_Validos) * 100
            print(f"  {i}. {Variable}: {Frecuencia} modelos ({Porcentaje:.1f}%)")

else:
    print(f"\n❌ No se pudieron generar modelos válidos")
    print(f"Revisar calidad de datos o especificación de variables")

print("="*60)


🔄 Procesando variable 1/9: CO_Item_6_Izq
Variable dependiente: CO_Item_6_Izq
Observaciones disponibles: 469
Valores faltantes en X antes de imputación:
  Conservative_Cluster: 3 → imputados con mediana
  Progressive_Cluster: 3 → imputados con mediana
Variables booleanas convertidas a enteros: 17

========================= ITERACIÓN 1 =========================
MODELO ROBUSTO METODOLÓGICAMENTE SÓLIDO: CO_Item_6_Izq
Variables en modelo actual: 50
Observaciones disponibles: 469

🔍 DETECCIÓN DE OUTLIERS INFLUYENTES:
  Outliers por Distancia de Cook (>0.0085): 46
  Outliers por Leverage (>0.2175): 21
  Outliers por Residuos (>|3.0|): 17
  Total outliers únicos detectados: 58
  ⚠️  58 observaciones influyentes detectadas
  ℹ️  No se eliminan automáticamente - evalúe manualmente

📊 DETECCIÓN DE MULTICOLINEALIDAD (VIF):
  Variables restantes: 50
  VIF máximo final: 3.05
  Variables eliminadas por VIF: 0

🔧 AJUSTANDO MODELO ROBUSTO:
  N observaciones: 469
  R²: 0.1363
  R² ajustado: 0.0330
  AI